## Neural networks intro

Early implementations of neural networks using self-implemented methods and comparing them to Keras/Tensorflow equivalents for the X detection problem

Testing points:

- Tensorflow model implementation
- Own NumPy Model implementation
- Vectorized Model implementation
- softmax and multiclass classification

In [9]:
import os,sys
module_path = os.path.abspath(os.path.join('..', 'jantools'))

if module_path not in sys.path:
    sys.path.append(module_path)
import importlib
import pandas as pd
import numpy as np
import json
import neural_networks_intro as nni

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [10]:
data = json.load(open("../data/neural_networks_intro.json"))

X = []
y = []

for item in data["images"]:
    X.append(item["pixels"])
    y.append(1 if item["label"] else 0)

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

X = X.reshape(X.shape[0], -1)
y = np.array(y, dtype=np.int32).reshape(-1, 1)

### Tensorflow

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from autils import *
%matplotlib inline

In [7]:
model = Sequential(
    [
        tf.keras.Input(shape=(400,)),

        Dense(25, activation="sigmoid"),
        Dense(15, activation="sigmoid"),
        Dense(1, activation="sigmoid"),

    ], name = "my_model"
)

model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 25)             │        10,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,431 (40.75 KB)

 Trainable params: 10,431 (40.75 KB)

 Non-trainable params: 0 (0.00 B)

#### Printing number of parameters for each layer

In [8]:
L1_num_params = 400 * 25 + 25
L2_num_params = 25 * 15 + 15
L3_num_params = 15 * 1 + 1

print(L1_num_params, L2_num_params, L3_num_params)

print("")
print("(w,b)")
print(model.layers[0].get_weights()[0].shape,model.layers[0].get_weights()[1].shape)
print(model.layers[1].get_weights()[0].shape,model.layers[1].get_weights()[1].shape)
print(model.layers[2].get_weights()[0].shape,model.layers[2].get_weights()[1].shape)

10025 390 16

(w,b)
(400, 25) (25,)
(25, 15) (15,)
(15, 1) (1,)


In [9]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X,y,
    epochs=20
)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 0.6807 
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.6049
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.5722
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.5536
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.5409
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.5283
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.5174
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.5068
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4959
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4848
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.4743
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4640
Epoch 13/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.4539
Epoch 14/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.4438
Epoch 15/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.4336
Epoch 16/20
4/4 ━━━━━━━━━━━━━━━━━

In [10]:
prediction = model.predict(X[0].reshape(1,400))  # an X
print(f" predicting an X: {prediction}")
prediction = model.predict(X[1].reshape(1,400))  # a non-X
print(f" predicting a non X:  {prediction}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
 predicting an X: [[0.6859828]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
 predicting a non X:  [[0.31907472]]


#### janmltools computations

In [21]:
import neural_networks_intro as nni
import time
importlib.reload(nni)
# We are using the same weights as computed by the tensorflow

[layer1, layer2, layer3] = model.layers
W1_tmp,b1_tmp = layer1.get_weights()
W2_tmp,b2_tmp = layer2.get_weights()
W3_tmp,b3_tmp = layer3.get_weights()

# prediction assesment and comparison between vectorized and non-vectorized versions

# Non-vectorized approach
start = time.perf_counter()
for i in range(100):
    prediction = nni.my_sequential(X[i], W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
    if prediction >= 0.5:
        yhat = 1
    else:
        yhat = 0
    #print( "yhat = ", yhat, " label= ", y[i,0])
end = time.perf_counter()
print(f"Elapsed time for non-vectorized: {end - start:.6f} seconds")

# Vectorized approach
start = time.perf_counter()
Prediction = nni.my_sequential_v(X, W1_tmp, b1_tmp, W2_tmp, b2_tmp, W3_tmp, b3_tmp )
end = time.perf_counter()
print(f"Elapsed time for vectorized: {end - start:.6f} seconds")


Elapsed time for non-vectorized: 0.028511 seconds
Elapsed time for vectorized: 0.004748 seconds


#### softmax and multiclass classification

In [31]:
data = json.load(open("../data/neural_networks_softmax.json"))

X = []
y = []

for item in data["samples"]:
    X.append(item["x"])
    y.append(item["y"])

X = np.array(X, dtype=np.float32)
y = np.array(y, dtype=np.int32)

X = X.reshape(X.shape[0], -1)
y = np.array(y, dtype=np.int32).reshape(-1, 1)

print(X.shape)
print(y.shape)

(100, 400)
(100, 1)


In [32]:
tf.random.set_seed(1234)
model = Sequential(
    [
        tf.keras.Input(shape=(400,)),

        Dense(25, activation='relu', name = "L1"),
        Dense(15, activation='relu', name = "L2"),
        Dense(10, activation='linear', name = "L3")

    ], name = "my_model"
)

In [33]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ L1 (Dense)                      │ (None, 25)             │        10,025 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L2 (Dense)                      │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ L3 (Dense)                      │ (None, 10)             │           160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,575 (41.31 KB)

 Trainable params: 10,575 (41.31 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
[layer1, layer2, layer3] = model.layers

W1,b1 = layer1.get_weights()
W2,b2 = layer2.get_weights()
W3,b3 = layer3.get_weights()
print(f"W1 shape = {W1.shape}, b1 shape = {b1.shape}")
print(f"W2 shape = {W2.shape}, b2 shape = {b2.shape}")
print(f"W3 shape = {W3.shape}, b3 shape = {b3.shape}")

W1 shape = (400, 25), b1 shape = (25,)
W2 shape = (25, 15), b2 shape = (15,)
W3 shape = (15, 10), b3 shape = (10,)


In [38]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
)

history = model.fit(
    X,y,
    epochs=40
)

Epoch 1/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 0.1812
Epoch 2/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1499
Epoch 3/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.1280
Epoch 4/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.1089
Epoch 5/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0926
Epoch 6/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0789
Epoch 7/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0674
Epoch 8/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0579
Epoch 9/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0500
Epoch 10/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0435
Epoch 11/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0380
Epoch 12/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0335
Epoch 13/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0296
Epoch 14/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0264
Epoch 15/40
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.0238
Epoch 16/40
4/4 ━━━━━━━━━━━━━━━━━━

In [44]:
image_of_two = X[0]
prediction = model.predict(image_of_two.reshape(1,400))
print(f" predicting a zero: ", np.argmax(prediction))

image_of_two = X[1]
prediction = model.predict(image_of_two.reshape(1,400))
print(f" predicting a two: ", np.argmax(prediction))

image_of_two = X[3]
prediction = model.predict(image_of_two.reshape(1,400))
print(f" predicting a five: ", np.argmax(prediction))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
 predicting a zero:  0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
 predicting a two:  2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
 predicting a five:  5
